In [1]:
import pandas as pd
import numpy as np

import lib.transformers as tf
from sklearn.pipeline import Pipeline

from lib.model import Ganomaly, GanomalyNet
from lib.visualization import GANomalyBoard

from skorch.callbacks import PassthroughScoring
import torch

from torch.utils.tensorboard import SummaryWriter

In [2]:
cwruData0 = pd.read_parquet('data/cwru0.parquet')
cwruData1 = pd.read_parquet('data/cwru1.parquet')

cwruData = pd.concat([cwruData0, cwruData1])

In [3]:
torch.manual_seed(0)
device = torch.device("cuda:0") if torch.cuda.is_available() else 'cpu'

In [4]:
nz = 100  # size of the latent z vector
ngf = 32  # units of generator
ndf = 32  # units of discriminator
nc = 1  # number of channels
batch_size = 32
lr = 0.0002
beta1 = 0.5  # for adam
max_epochs = 50
ngpu = 1
isize = 32  # 32 is easier than 28 to work with
workers = 2  # for dataloader

In [5]:
net = GanomalyNet(
    Ganomaly,
    module__isize = isize,
    module__nz=nz,
    module__ndf=ndf,
    module__ngf=ngf,
    module__nc=nc,
    module__ngpu=ngpu,
    
    module__w_lat = 1,
    
    criterion=torch.nn.BCELoss,

    optimizer_gen=torch.optim.Adam,
    optimizer_gen__lr=0.0002,
    optimizer_gen__betas=(beta1, 0.999),

    optimizer_dis=torch.optim.Adam,
    optimizer_dis__lr=0.00002,
    optimizer_dis__betas=(beta1, 0.999),

    batch_size=batch_size,
    max_epochs=max_epochs,

    train_split=False,  # not implemented
    iterator_train__shuffle=True,
    iterator_train__num_workers=workers,
    iterator_valid__num_workers=workers,

    callbacks=[
        PassthroughScoring('loss_dis', on_train=True),
        PassthroughScoring('loss_gen', on_train=True),
        PassthroughScoring('loss_gen_fra', on_train=True),
        PassthroughScoring('loss_gen_app', on_train=True),
        PassthroughScoring('loss_gen_lat', on_train=True),
        GANomalyBoard(SummaryWriter())
        
    ],
)

In [6]:
normal = tf.DataSelector(columns = ['fanEndData', 'driveEndData'], condition = ['Normal Baseline']).transform(cwruData)
inner = tf.DataSelector(columns = ['fanEndData', 'driveEndData'], condition = ['Inner Race Fault']).transform(cwruData)
ball = tf.DataSelector(columns = ['fanEndData', 'driveEndData'], condition = ['Ball Fault']).transform(cwruData)
outer = tf.DataSelector(columns = ['fanEndData', 'driveEndData'], condition = ['Outer Race Fault']).transform(cwruData)

In [7]:
params = {
    'module__w_fra': [10, 12],
}

from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(net, params, refit=False, cv=3)
from sklearn.model_selection import GridSearchCV

prep = \
Pipeline(steps=[
                ('ArrayFlattener', tf.ArrayFlattener()),
                ('DatatypeFilter', tf.DataTypeFilter(data_type = np.ndarray)),
                ('ArrayChunker', tf.ArrayChunker(1024)),
                #('ArrayEqualizer', tf.ArrayEqualizer()),
                ('ArrayFlattener2', tf.ArrayFlattener()),
                ('RandomArraySampler', tf.RandomArraySampler(400, random_state = 0)),
                #('FeatureExtractor', tf.FeatureExtractor(axis=1)),
                ('ArrayReshaper', tf.ArrayReshaper((1, 32, 32))),
                ('GANomaly', net)
               ])

test = prep.fit(normal)

C:\Users\jlsachse\Documents\Bachelorarbeit\Implementation\ganomaly\lib\transformers.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_ = np.array(X_)
C:\Users\jlsachse\Documents\Bachelorarbeit\Implementation\ganomaly\lib\transformers.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_ = np.array(X_)


  epoch    loss_dis    loss_gen    loss_gen_app    loss_gen_fra    loss_gen_lat    train_loss     dur
-------  ----------  ----------  --------------  --------------  --------------  ------------  ------
      1      0.6507      1.2140          0.1152          0.6985          0.4003        1.8648  6.9934
      2      0.6408      0.8516          0.1033          0.6075          0.1407        1.4924  6.0772
      3      0.6299      0.7062          0.0911          0.5281          0.0870        1.3361  5.9698
      4      0.6177      0.6032          0.0832          0.4585          0.0614        1.2209  6.3632
      5      0.6038      0.5281          0.0778          0.3971          0.0532        1.1319  6.8899
      6      0.5887      0.4592          0.0741          0.3430          0.0421        1.0479  8.1055
      7      0.5719      0.4047          0.0711          0.2954          0.0381        0.9765  6.7771
      8      0.5523      0.3585          0.0693          0.2535          0.0356   

In [8]:
print("normal mean:", prep.predict(normal).mean())
print("inner mean:", prep.predict(inner).mean())
print("ball mean:", prep.predict(ball).mean())
print("outer mean:", prep.predict(outer).mean())

normal mean: 0.030068059
inner mean: 0.85998094
ball mean: 0.21447766
outer mean: 1.155004
